# Data loading

# Python packages




In [1]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


# Door open - inbound




In [4]:
inbound = pd.read_csv('inbound_loads.csv')
outbound = pd.read_csv('outbound_laods.csv')
ft = pd.read_csv('incoming_weight_df_10h.csv')
#ft = pd.read_csv('featuretrain.csv')

### Uitleg

1. Get all the minutes for the interval in between start and finish


In [5]:
def interval_min(start_date, end_date):
    dates_range = []
    while start_date <= end_date:
        dates_range.append(start_date) # or you can append str(start_date) if want the dates to be as strings
        #dates_range.append(start_date - timedelta(seconds=start_date.second) + timedelta(minutes=1)) # or you can append str(start_date) if want the dates to be as strings
        start_date += timedelta(minutes=1)
    return dates_range

In [6]:
door = inbound.filter(['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], axis=1)
door

,load_start_datetime,load_finish_datetime,dock_door_number
0,2019-02-11 04:42:00,2019-02-11 04:51:00,59.0
1,2019-01-16 10:11:00,2019-01-16 11:27:00,60.0
2,2019-07-30 09:59:00,2019-07-30 10:18:00,61.0
3,2019-01-10 08:51:00,2019-01-10 08:58:00,58.0
4,2019-01-10 07:26:00,2019-01-10 08:08:00,56.0
...,...,...,...
56141,NaN,NaN,NaN
56142,NaN,NaN,NaN
56143,NaN,NaN,NaN
56144,2022-01-03 16:12:00,2022-01-03 16:26:00,56.0


2. Get all the minutes intervals and put them in a new dataframe

In [7]:
door['load_finish_datetime'] = pd.to_datetime(door['load_finish_datetime'])
door['load_start_datetime'] = pd.to_datetime(door['load_start_datetime'])

door['load_minutes'] = np.nan

for index, row in door.iterrows():
    door['load_minutes'].iloc[index] = interval_min(door['load_start_datetime'].iloc[index], door['load_finish_datetime'].iloc[index])

C:\Users\suzan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
door = door.drop_duplicates(subset=['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], keep="first")

In [9]:
door

,load_start_datetime,load_finish_datetime,dock_door_number,load_minutes
0,2019-02-11 04:42:00,2019-02-11 04:51:00,59.0,"[2019-02-11 04:42:00, 2019-02-11 04:43:00, 201..."
1,2019-01-16 10:11:00,2019-01-16 11:27:00,60.0,"[2019-01-16 10:11:00, 2019-01-16 10:12:00, 201..."
2,2019-07-30 09:59:00,2019-07-30 10:18:00,61.0,"[2019-07-30 09:59:00, 2019-07-30 10:00:00, 201..."
3,2019-01-10 08:51:00,2019-01-10 08:58:00,58.0,"[2019-01-10 08:51:00, 2019-01-10 08:52:00, 201..."
4,2019-01-10 07:26:00,2019-01-10 08:08:00,56.0,"[2019-01-10 07:26:00, 2019-01-10 07:27:00, 201..."
...,...,...,...,...
56135,2022-01-02 21:07:00,2022-01-02 21:20:00,54.0,"[2022-01-02 21:07:00, 2022-01-02 21:08:00, 202..."
56137,2022-01-03 00:38:00,2022-01-03 00:45:00,68.0,"[2022-01-03 00:38:00, 2022-01-03 00:39:00, 202..."
56138,2022-01-03 06:45:00,2022-01-03 06:57:00,42.0,"[2022-01-03 06:45:00, 2022-01-03 06:46:00, 202..."
56144,2022-01-03 16:12:00,2022-01-03 16:26:00,56.0,"[2022-01-03 16:12:00, 2022-01-03 16:13:00, 202..."


3. Make a flat list from the 'load_minutes' column 

In [10]:
def flatten(l):
    return [item for sublist in l for item in sublist]

list_min = door['load_minutes'].tolist()
list_min = flatten(list_min)
type(list_min[0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
list_min

[Timestamp('2019-02-11 04:42:00'),
 Timestamp('2019-02-11 04:43:00'),
 Timestamp('2019-02-11 04:44:00'),
 Timestamp('2019-02-11 04:45:00'),
 Timestamp('2019-02-11 04:46:00'),
 Timestamp('2019-02-11 04:47:00'),
 Timestamp('2019-02-11 04:48:00'),
 Timestamp('2019-02-11 04:49:00'),
 Timestamp('2019-02-11 04:50:00'),
 Timestamp('2019-02-11 04:51:00'),
 Timestamp('2019-01-16 10:11:00'),
 Timestamp('2019-01-16 10:12:00'),
 Timestamp('2019-01-16 10:13:00'),
 Timestamp('2019-01-16 10:14:00'),
 Timestamp('2019-01-16 10:15:00'),
 Timestamp('2019-01-16 10:16:00'),
 Timestamp('2019-01-16 10:17:00'),
 Timestamp('2019-01-16 10:18:00'),
 Timestamp('2019-01-16 10:19:00'),
 Timestamp('2019-01-16 10:20:00'),
 Timestamp('2019-01-16 10:21:00'),
 Timestamp('2019-01-16 10:22:00'),
 Timestamp('2019-01-16 10:23:00'),
 Timestamp('2019-01-16 10:24:00'),
 Timestamp('2019-01-16 10:25:00'),
 Timestamp('2019-01-16 10:26:00'),
 Timestamp('2019-01-16 10:27:00'),
 Timestamp('2019-01-16 10:28:00'),
 Timestamp('2019-01-

4. Create new dataframe with each time as an individual row (instead of list in each row). And change type from timeFrame to Datetime

In [12]:
minutes = pd.DataFrame(list_min)
minutes.columns =['datetime_local']
#minutes['datetime_local'] = pd.to_datetime(minutes['datetime_local'], unit='s')
#minutes['datetime_local'] = pd.Series(minutes.datetime_local.dt.to_pydatetime(), dtype='O')
minutes

,datetime_local
0,2019-02-11 04:42:00
1,2019-02-11 04:43:00
2,2019-02-11 04:44:00
3,2019-02-11 04:45:00
4,2019-02-11 04:46:00
...,...
1559988,2022-01-03 16:22:00
1559989,2022-01-03 16:23:00
1559990,2022-01-03 16:24:00
1559991,2022-01-03 16:25:00


5. Create a new dataframe with datetime and count

In [13]:
test = minutes.value_counts().to_frame('counts')
#test = minutes.value_counts().rename_axis('datetime_local').to_frame('counts')
test

,counts
datetime_local,
2021-12-23 09:48:00,12
2021-12-23 09:50:00,12
2021-12-23 09:51:00,12
2021-12-23 09:52:00,12
2021-12-23 09:53:00,12
...,...
2020-04-10 17:55:00,1
2020-04-10 17:56:00,1
2020-04-10 17:57:00,1


In [14]:
test['counts'].unique()

array([12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1], dtype=int64)

Bit sketchy but works.
- Reset the index level of dataframe 'test' and then turn Timeframe type into datetime
- Do the same with dataframe 'ft', so reset index level (is string), then turn type to datetime.


In [15]:
test = test.reset_index(level=[0])
#type(test.index)
#type(test['datetime_local'][0])

test['datetime_local'] = pd.to_datetime(test['datetime_local'], unit='s')
test['datetime_local'] = pd.Series(test.datetime_local.dt.to_pydatetime(), dtype='O')

In [16]:
type(test.datetime_local[0])

datetime.datetime

In [17]:
ft = ft[2326:]

In [18]:
ft = ft.reset_index(level=[0])
type(ft['datetime_local'][0])
ft = ft.drop(['index', 'weight_23h'], axis=1)
ft.head()

,datetime_local
0,2018-12-31 21:15:00
1,2018-12-31 21:16:00
2,2018-12-31 21:17:00
3,2018-12-31 21:18:00
4,2018-12-31 21:19:00


In [19]:
ft

,datetime_local
0,2018-12-31 21:15:00
1,2018-12-31 21:16:00
2,2018-12-31 21:17:00
3,2018-12-31 21:18:00
4,2018-12-31 21:19:00
...,...
1585596,2022-01-04 03:53:00
1585597,2022-01-04 03:54:00
1585598,2022-01-04 03:55:00
1585599,2022-01-04 03:56:00


In [20]:
ft['datetime_local'] = pd.to_datetime(ft['datetime_local'])
ft['datetime_local'] = pd.Series(ft.datetime_local.dt.to_pydatetime(), dtype='O')
type(ft.datetime_local[0])

datetime.datetime

- Set datetime_local as index again. (Now the type of both is the same, which is necessary for merging correctly).

In [21]:
ft.set_index('datetime_local', inplace=True)
test.set_index('datetime_local', inplace=True)

- Now merge ft and test dataframe, on the content of the datetime_local of the ft dataframe. 


In [ ]:
#2020-02-08 07:03:34

In [ ]:
#hi = pd.concat([ft, test], axis=1)


In [22]:
feat_min = ft.merge(test, left_on='datetime_local', right_index=True, how='left').reset_index()

In [23]:
feat_min.iloc[584209]

datetime_local    2020-02-09 21:52:00
counts                            NaN
Name: 584209, dtype: object

In [24]:
feat_min

,datetime_local,counts
0,2018-12-31 21:15:00,NaN
1,2018-12-31 21:16:00,NaN
2,2018-12-31 21:17:00,NaN
3,2018-12-31 21:18:00,NaN
4,2018-12-31 21:19:00,NaN
...,...,...
1585596,2022-01-04 03:53:00,NaN
1585597,2022-01-04 03:54:00,NaN
1585598,2022-01-04 03:55:00,NaN
1585599,2022-01-04 03:56:00,NaN


In [26]:
feat_min["counts"] = feat_min["counts"].fillna(0)

#Drop certain columns if necessary
#feat_min = feat_min.drop(['level_0_x', 'index_x','level_0_y', 'index_y'], axis=1)
#feat_min.to_csv("feat_door_open")

In [27]:
feat_min['counts'].unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

In [28]:
feat_min.set_index('datetime_local', inplace=True)
#feat_min = feat_min.drop(['level_0', 'index','year', 'month','weekday', 'day', 'hour', 'minute', 'datetime'], axis=1)
#feat_min = feat_min.drop(['Temperature', 'Relative Humidity', 'datetime_UTC'], axis=1)
feat_min


,counts
datetime_local,
2018-12-31 21:15:00,0.0
2018-12-31 21:16:00,0.0
2018-12-31 21:17:00,0.0
2018-12-31 21:18:00,0.0
2018-12-31 21:19:00,0.0
...,...
2022-01-04 03:53:00,0.0
2022-01-04 03:54:00,0.0
2022-01-04 03:55:00,0.0


In [ ]:
feat_min.iloc[400]

In [29]:
feat_min.to_csv("feature_inbound_door_open.csv")